# Assigment 7

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

### 1.Import data from PDF Format

In [8]:
!pip install PyPDF2

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3


In [166]:
import pandas as pd
from functools import reduce

In [95]:
## Link 
link= 'https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf'

In [432]:
import io
import requests
import PyPDF2

r = requests.get(link)
f = io.BytesIO(r.content)

reader = PyPDF2.PdfReader(f)
page_content=[] 
for i in range(len(reader.pages)):
    page_content +=reader.pages[i].extract_text().strip().split('   ')

res_ica = list(map(lambda st: str.replace(st, "ICA ", "ICA\t"), page_content))
res_t = list(map(lambda st: str.replace(st, "  ", "\t"), res_ica))
res2 = reduce(lambda x, y: x +"\t"+ y, res_t)
res_per = res2. replace( "PASEO DE LA\tREPÚBLICA\tS/N - CCPLAZA LIMA SUR",
                                            "PASEO DE LA REPÚBLICA S/N - CCPLAZA LIMA SUR") 

res_ca = res_per.replace("CA.\tDERECHA\t370","CA. DERECHA 370")
res_may = res_ca.replace("CALLE DOS\tDE MAYO Nº\t9 -PACASMAYO","CALLE DOS DE MAYO Nº 9 -PACASMAYO")
res_3 = res_may.replace("URB. CACHIMAYO\tC -21-A\tSAN SEBASTIAN","URB. CACHIMAYO C -21-A SAN SEBASTIAN")
res_4 = res_3.replace("CARRETERA\tPANAMERICANA\tURB. SANTO DOMINGO MZ C LOTE",
                        "CARRETERA PANAMERICANA URB.SANTO DOMINGO MZ C LOTE")
res_5 = res_4.replace("\tCANGALLO N. - 180 LA VICTORIA",
                        "CANGALLO N. - 180\tLA VICTORIA")
res_6 = res_5.replace("AV. PASEO LA REPÚBLICA\tS/N Y CRUCE CON AV. BOLIVIA S/N, LC \nLS-15 LIMA",
                        "AV. PASEO LA REPÚBLICA S/N Y CRUCE CON AV. BOLIVIA S/N, LC \nLS-15\tLIMA")

res_vf = res_6.replace("\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t", "")
res_ofi = res_vf.replace("\tOFICINA", "\t\nOFICINA")
base_to_df =res_ofi.strip().split('\t\n')
res_f = list(map(lambda st: str.replace(st, "\n", ""), base_to_df))


df = pd.DataFrame(res_f, columns = ['bbva_sedes'])
df_ini = df['bbva_sedes'].str.split('\t', expand=True)
df_sec = df['bbva_sedes'].str.split('\t', expand=True).dropna()
lista_sec =list(df_sec.index.values)+[0]
df_2 = df_ini.drop(lista_sec)
df_2.rename(columns={0: 'OFICINA', 1: 'DIRECCION', 2 :'DISTRITO',3: 'PROVINCIA', 4: 'DEPARTAMENTO'}, inplace=True)
df_3 = df_2.drop(columns=5)
df_sec[6] = df_sec[1]+" "+df_sec[2]  
df_sec_1 = df_sec.drop(columns=[1,2])
df_sec_1.rename(columns={0: 'OFICINA', 6: 'DIRECCION', 3 :'DISTRITO',4: 'PROVINCIA', 5: 'DEPARTAMENTO'}, inplace=True)
Df_oficinas = pd.concat([df_3,df_sec_1]).reset_index().drop(columns='index')
Df_oficinas_Lima = Df_oficinas[Df_oficinas["DEPARTAMENTO"]=="LIMA"].reset_index().drop(columns='index')

Overwriting cache for 0 3777


In [434]:
Df_oficinas_Lima

,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO
0,CAMINO REAL,AV. CAMINO REAL 355,SAN ISIDRO,LIMA,LIMA
1,28 DE JULIO,AV. 28 DE JULIO 775,LIMA,LIMA,LIMA
2,AEROPUERTO,CENTRO AÉREO COMERCIAL LOCALES 110 A Y 111 A,CALLAO,LIMA,LIMA
3,ANGAMOS ESTE,"Av. Angamos Este 215 - 227, esquina con Av. Pe...",MIRAFLORES,LIMA,LIMA
4,APOLO,AV. MEXICO 1700,LA VICTORIA,LIMA,LIMA
...,...,...,...,...,...
164,V. A. BELAUNDE,CA. VICTOR A. BELAUNDE ESTE 101 URB. POPULAR R...,COMAS,LIMA,LIMA
165,VARGAS MACHUCA,"AV. VARGAS MACHUCA 475, ESQ. PEDRO BERTONELLI",SAN JUAN DE MIRAFLORES,LIMA,LIMA
166,VILLA MARIA DEL TRIUNFO,"AV. PACHACÚTEC 2101, PRIMER HOGAR POLICIAL, AS...",VILLA MARIA DEL TRIUNFO,LIMA,LIMA
167,EMBAJADA USA,AV. LA ENCALADA S/N -EMBAJADA ESTADOS UNIDOS.,SANTIAGO DE SURCO,LIMA,LIMA


In [436]:
Df_oficinas_Lima.groupby('DISTRITO').count()

,OFICINA,DIRECCION,PROVINCIA,DEPARTAMENTO
DISTRITO,,,,
SAN JUAN DE LURIGANCHO,1,1,1,1
SAN MIGUEL,1,1,1,1
BARRANCA,2,2,2,2
BARRANCO,2,2,2,2
BELLAVISTA,1,1,1,1
BREÑA,3,3,3,3
CALLAO,10,10,10,10
CARABAYLLO,1,1,1,1
CHANCAY,1,1,1,1
